In [4]:
def run_chronospatial_computer(A_init, B_init, C_init, program):
    """
    Runs the 3-bit Chronospatial Computer with the given registers and program.
    Returns a list of output values (the 'out' instruction results).
    """
    A = A_init
    B = B_init
    C = C_init

    def combo_value(operand):
        # combo operands: 0..3 => literal, 4 => A, 5 => B, 6 => C, 7 => invalid
        if operand < 4:
            return operand
        elif operand == 4:
            return A
        elif operand == 5:
            return B
        elif operand == 6:
            return C
        else:
            # operand == 7 shouldn't happen in valid programs
            raise ValueError("Invalid combo operand: 7")

    outputs = []
    ip = 0  # instruction pointer

    while ip < len(program):
        opcode = program[ip]
        if ip + 1 < len(program):
            operand = program[ip + 1]
        else:
            # If there's no operand for the last opcode, we halt
            break

        if opcode == 0:
            # adv: A = A // (2^(combo operand))
            val = combo_value(operand)
            A //= (2 ** val)
            ip += 2

        elif opcode == 1:
            # bxl: B = B ^ operand (literal)
            B ^= operand
            ip += 2

        elif opcode == 2:
            # bst: B = combo_value(operand) % 8
            B = combo_value(operand) % 8
            ip += 2

        elif opcode == 3:
            # jnz: if A != 0, jump to operand (literal)
            if A != 0:
                ip = operand
            else:
                ip += 2

        elif opcode == 4:
            # bxc: B = B ^ C   (operand ignored)
            B ^= C
            ip += 2

        elif opcode == 5:
            # out: outputs combo_value(operand) % 8
            val = combo_value(operand)
            outputs.append(val % 8)
            ip += 2

        elif opcode == 6:
            # bdv: B = A // (2^(combo operand))
            val = combo_value(operand)
            B = A // (2 ** val)
            ip += 2

        elif opcode == 7:
            # cdv: C = A // (2^(combo operand))
            val = combo_value(operand)
            C = A // (2 ** val)
            ip += 2

        else:
            # invalid opcode -> halt
            break

    return outputs


def program_matches_self(A_candidate, B_init, C_init, program):
    """Returns True if, with A=A_candidate, the program outputs exactly `program`."""
    outputs = run_chronospatial_computer(A_candidate, B_init, C_init, program)
    return outputs == program


def main():
    # Read from input.txt
    with open('input.txt', 'r') as f:
        lines = [line.strip() for line in f if line.strip()]

    B_init = 0
    C_init = 0
    program_str = None

    for line in lines:
        # We ignore "Register A" because puzzle states it's corrupted and we're finding a replacement A.
        if line.startswith("Register B:"):
            B_init = int(line.split("Register B:")[1].strip())
        elif line.startswith("Register C:"):
            C_init = int(line.split("Register C:")[1].strip())
        elif line.startswith("Program:"):
            program_str = line.split("Program:")[1].strip()

    program = list(map(int, program_str.split(',')))
    # We want the output of the program to EXACTLY match `program`.

    # Exponential search to find an upper bound for A
    low = 1
    high = 1
    found_range = False

    while True:
        if program_matches_self(high, B_init, C_init, program):
            # Found a solution at 'high', so the answer is in [low, high]
            found_range = True
            break
        # If not found, move to next power of 2.  But if 'high' is huge, watch out for overflow...
        if high > 1*10**100:
            # Safety cutoff: if the puzzle requires a bigger A than that, we might suspect
            # there's no solution or the puzzle is extremely large. 
            break
        low = high + 1
        high *= 2  # Exponential growth

    if not found_range:
        # If we never found an upper bound <= 1_000_000_000, either the solution is bigger or doesn't exist.
        print("No solution found up to 1,000,000,000. (Increase the limit or do deeper analysis.)")
        return

    # Now we do a binary search in [low, high] for the smallest solution
    answer = high
    left, right = low, high
    while left <= right:
        mid = (left + right) // 2
        if program_matches_self(mid, B_init, C_init, program):
            answer = mid
            right = mid - 1
        else:
            left = mid + 1

    print(answer)


if __name__ == "__main__":
    main()

No solution found up to 1,000,000,000. (Increase the limit or do deeper analysis.)


In [1]:
def run_chronospatial_computer(A_init, B_init, C_init, program):
    """
    Runs the 3-bit Chronospatial Computer with the given registers and program.
    Returns a list of output values (the 'out' instruction results).
    """
    A = A_init
    B = B_init
    C = C_init

    def combo_value(operand):
        # combo operands: 0..3 => literal, 4 => A, 5 => B, 6 => C, 7 => invalid
        if operand < 4:
            return operand
        elif operand == 4:
            return A
        elif operand == 5:
            return B
        elif operand == 6:
            return C
        else:
            # operand == 7 shouldn't happen in valid programs
            raise ValueError("Invalid combo operand: 7")

    outputs = []
    ip = 0  # instruction pointer

    while ip < len(program):
        opcode = program[ip]
        if ip + 1 < len(program):
            operand = program[ip + 1]
        else:
            # If there's no operand for the last opcode, we halt
            break

        if opcode == 0:
            # adv: A = A // (2^(combo operand))
            val = combo_value(operand)
            A //= (2 ** val)
            ip += 2

        elif opcode == 1:
            # bxl: B = B ^ operand (literal)
            B ^= operand
            ip += 2

        elif opcode == 2:
            # bst: B = combo_value(operand) % 8
            B = combo_value(operand) % 8
            ip += 2

        elif opcode == 3:
            # jnz: if A != 0, jump to operand (literal)
            if A != 0:
                ip = operand
            else:
                ip += 2

        elif opcode == 4:
            # bxc: B = B ^ C   (operand ignored)
            B ^= C
            ip += 2

        elif opcode == 5:
            # out: outputs combo_value(operand) % 8
            val = combo_value(operand)
            outputs.append(val % 8)
            ip += 2

        elif opcode == 6:
            # bdv: B = A // (2^(combo operand))
            val = combo_value(operand)
            B = A // (2 ** val)
            ip += 2

        elif opcode == 7:
            # cdv: C = A // (2^(combo operand))
            val = combo_value(operand)
            C = A // (2 ** val)
            ip += 2

        else:
            # invalid opcode -> halt
            break

    return outputs


def program_matches_self(A_candidate, B_init, C_init, program):
    """Returns True if, with A=A_candidate, the program outputs exactly `program`."""
    outputs = run_chronospatial_computer(A_candidate, B_init, C_init, program)
    return outputs == program


def main():
    # Read from input.txt
    with open('input.txt', 'r') as f:
        lines = [line.strip() for line in f if line.strip()]

    B_init = 0
    C_init = 0
    program_str = None

    for line in lines:
        # We ignore "Register A" because puzzle states it's corrupted and we're finding a replacement A.
        if line.startswith("Register B:"):
            B_init = int(line.split("Register B:")[1].strip())
        elif line.startswith("Register C:"):
            C_init = int(line.split("Register C:")[1].strip())
        elif line.startswith("Program:"):
            program_str = line.split("Program:")[1].strip()

    program = list(map(int, program_str.split(',')))
    # We want the output of the program to EXACTLY match `program`.

    # Exponential search to find an upper bound for A
    low = 1
    high = 1
    found_range = False

    while True:
        if program_matches_self(high, B_init, C_init, program):
            # Found a solution at 'high', so the answer is in [low, high]
            found_range = True
            break
        # If not found, move to next power of 2.  But if 'high' is huge, watch out for overflow...
        if high > 1*10**100:
            # Safety cutoff: if the puzzle requires a bigger A than that, we might suspect
            # there's no solution or the puzzle is extremely large. 
            break
        low = high + 1
        high *= 2  # Exponential growth

    if not found_range:
        # If we never found an upper bound <= 1_000_000_000, either the solution is bigger or doesn't exist.
        print("No solution found up to 1,000,000,000. (Increase the limit or do deeper analysis.)")
        return

    # Now we do a binary search in [low, high] for the smallest solution
    answer = high
    left, right = low, high
    while left <= right:
        mid = (left + right) // 2
        if program_matches_self(mid, B_init, C_init, program):
            answer = mid
            right = mid - 1
        else:
            left = mid + 1

    print(answer)


if __name__ == "__main__":
    main()

No solution found up to 1,000,000,000. (Increase the limit or do deeper analysis.)


In [16]:
import sys
import z3
from z3 import Solver, Int, And, Or, If, sat, Optimize

def parse_input(filename='input.txt'):
    """Parses the input.txt file and returns (B_init, C_init, program)."""
    with open(filename, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]

    B_init = 0
    C_init = 0
    program_str = None

    for line in lines:
        if line.startswith("Register B:"):
            B_init = int(line.split("Register B:")[1].strip())
        elif line.startswith("Register C:"):
            C_init = int(line.split("Register C:")[1].strip())
        elif line.startswith("Program:"):
            program_str = line.split("Program:")[1].strip()

    program = list(map(int, program_str.split(',')))
    return B_init, C_init, program

def combo_value(operand, A_val, B_val, C_val):
    """
    Returns a Z3 expression for the "combo operand" value.
    0..3 => literal, 4 => A, 5 => B, 6 => C
    """
    return If(operand < 4,
              operand,
              If(operand == 4, A_val,
                 If(operand == 5, B_val,
                    If(operand == 6, C_val, -999999999))))  # unreachable default

def main():
    B_init, C_init, program = parse_input('input.txt')
    prog_len = len(program)
    max_steps = 2 * prog_len

    s = Solver()

    A_init = Int('A_init')
    s.add(A_init > 0)

    A_vars = [Int(f"A_{i}") for i in range(max_steps + 1)]
    B_vars = [z3.BitVec(f"B_{i}", 32) for i in range(max_steps + 1)]

    C_vars = [Int(f"C_{i}") for i in range(max_steps + 1)]
    ip_vars = [Int(f"ip_{i}") for i in range(max_steps + 1)]
    out_idx_vars = [Int(f"out_idx_{i}") for i in range(max_steps + 1)]
    outs = [Int(f"outs_{i}") for i in range(prog_len)]

    for i in range(prog_len):
        s.add(outs[i] == program[i])

    s.add(A_vars[0] == A_init)
    s.add(B_vars[0] == B_init)
    s.add(C_vars[0] == C_init)
    s.add(ip_vars[0] == 0)
    s.add(out_idx_vars[0] == 0)

    for step in range(max_steps):
        A_cur, B_cur, C_cur, ip_cur, outi_cur = A_vars[step], B_vars[step], C_vars[step], ip_vars[step], out_idx_vars[step]
        A_next, B_next, C_next, ip_next, outi_next = A_vars[step+1], B_vars[step+1], C_vars[step+1], ip_vars[step+1], out_idx_vars[step+1]

        halt_cond = ip_cur >= prog_len

        # Safely extract opcode and operand
        opcode = -1
        operand = 0
        for i in range(prog_len):
            opcode = If(ip_cur == i, program[i], opcode)
            operand = If(ip_cur == i, program[i + 1] if i + 1 < prog_len else 0, operand)

        # Transition logic
        A_n, B_n, C_n, ip_n, outi_n = A_cur, B_cur, C_cur, ip_cur + 2, outi_cur



        A_n = If(opcode == 0, A_cur / (2 ** combo_value(operand, A_cur, B_cur, C_cur)), A_n)



        B_n = If(opcode == 1, B_cur ^ z3.BitVecVal(operand, 32), B_n)




        B_n = If(opcode == 2, combo_value(operand, A_cur, B_cur, C_cur) % 8, B_n)
        ip_n = If(opcode == 3, If(A_cur != 0, operand, ip_cur + 2), ip_n)
        B_n = If(opcode == 4, B_cur ^ C_cur, B_n)
        outi_n = If(opcode == 5, outi_cur + 1, outi_n)

        s.add(If(halt_cond,
                 And(A_next == A_cur, B_next == B_cur, C_next == C_cur, ip_next == ip_cur, outi_next == outi_cur),
                 And(A_next == A_n, B_next == B_n, C_next == C_n, ip_next == ip_n, outi_next == outi_n)
                 ))

    s.add(out_idx_vars[max_steps] == prog_len)
    s.add(ip_vars[max_steps] >= prog_len)

    result = s.check()
    if result == sat:
        model = s.model()
        opt = Optimize()
        opt.add(s.assertions())
        h = opt.minimize(A_init)
        if opt.check() == sat:
            print("Minimal A:", opt.model()[A_init])
        else:
            print("No minimal solution found")
    else:
        print("No solution found")


if __name__ == "__main__":
    main()


Z3Exception: sort mismatch